In [2]:
import json, math, glob
from collections import Counter

import numpy as np
import pandas as pd
import scipy.stats as stats
from tqdm import tqdm


import plotly.express as px
import plotly.graph_objects as go

from arena import result_table, pass1_to_battle 

records = []
for fname in glob.glob(f"data/*.jsonl"):
    with open(fname, 'rt') as f:
        records.extend([json.loads(l) for l in f.readlines()])

eval_results = pd.DataFrame(records)


In [38]:
import importlib
import arena
importlib.reload(arena)

def subsample(results, n=100):
    eids = set(results['example_id'])
    include_ids = np.random.choice(list(eids), n, replace=False)
    return results[results['example_id'].isin(include_ids)]

def sample_table(results):
    results = subsample(results, 164)
    battles = pass1_to_battle(results)
    # battles= battles[battles["winner"].str.contains("model_")]
    result_tbl = arena.result_table(battles, results)
    return result_tbl

for b in ['humaneval+']:
    results = eval_results[eval_results['benchmark_id'] == b]
    result1 = sample_table(results)
    display(result1)
    # result2 = sample_table(results)
    # result_tblo = result1.merge(result2, on='model', suffixes=['_1', '_2'])
    # display(result_tblo)


,model,pass1,win_rate,elo
4,claude-3-opus-20240229,0.774390,0.252795,1074.682237
17,deepseek-coder-33b-instruct,0.762195,0.241489,1070.180987
32,opencodeinterpreter-ds-33b,0.743902,0.238313,1063.469311
27,mixtral-8x22b-instruct-v0.1,0.737805,0.233740,1061.242032
37,speechless-codellama-34b,0.725610,0.219893,1056.801266
0,HuggingFaceH4--starchat2-15b-v0.1,0.719512,0.219258,1054.587410
11,code-millenials-34b,0.719512,0.222815,1054.587410
18,deepseek-coder-6.7b-instruct,0.719512,0.232342,1054.587410
22,meta-llama-3-70b-instruct,0.719512,0.219385,1054.587410
19,deepseek-coder-7b-instruct-v1.5,0.713415,0.216336,1052.377664


In [40]:
result_tbl = result_tblo.set_index('model')
display(result_tbl.corr())
for v1 in result_tbl.columns:
    for v2 in result_tbl.columns:
        kt = stats.kendalltau(result_tbl[v1], result_tbl[v2])
        print(v1, v2, kt.statistic)


,pass1_1,win_rate_1,elo_1,pass1_2,win_rate_2,elo_2
pass1_1,1.000000,0.997382,0.997442,0.974997,0.969106,0.962326
win_rate_1,0.997382,1.000000,0.998887,0.974305,0.974776,0.966176
elo_1,0.997442,0.998887,1.000000,0.973309,0.971976,0.965488
pass1_2,0.974997,0.974305,0.973309,1.000000,0.994639,0.994762
win_rate_2,0.969106,0.974776,0.971976,0.994639,1.000000,0.997025
elo_2,0.962326,0.966176,0.965488,0.994762,0.997025,1.000000


pass1_1 pass1_1 1.0
pass1_1 win_rate_1 0.9755612440507009
pass1_1 elo_1 0.9721231427500816
pass1_1 pass1_2 0.8849327799071217
pass1_1 win_rate_2 0.8569467491793381
pass1_1 elo_2 0.8586657998296476
win_rate_1 pass1_1 0.9755612440507009
win_rate_1 win_rate_1 0.9999999999999998
win_rate_1 elo_1 0.9965986394557822
win_rate_1 pass1_2 0.8797718387651441
win_rate_1 win_rate_2 0.8554421768707482
win_rate_1 elo_2 0.857142857142857
elo_1 pass1_1 0.9721231427500816
elo_1 win_rate_1 0.9965986394557822
elo_1 elo_1 0.9999999999999998
elo_1 pass1_2 0.8797718387651441
elo_1 win_rate_2 0.8554421768707482
elo_1 elo_2 0.857142857142857
pass1_2 pass1_1 0.8849327799071218
pass1_2 win_rate_1 0.8797718387651441
pass1_2 elo_1 0.8797718387651441
pass1_2 pass1_2 1.0
pass1_2 win_rate_2 0.9708419705123171
pass1_2 elo_2 0.9691236661397291
win_rate_2 pass1_1 0.8569467491793381
win_rate_2 win_rate_1 0.8554421768707482
win_rate_2 elo_1 0.8554421768707482
win_rate_2 pass1_2 0.9708419705123171
win_rate_2 win_rate_2 0.9

In [ ]:
from scipy.integrate import dblquad
from scipy.special import gamma

def beta_n(x, ax, bx):
    return gamma(ax + bx) / gamma(ax) / gamma(bx) * x**(ax-1) * (1-x)**(bx-1) 
def beta_coef(y, x, ax, bx, ay, by):
    return beta_n(x, ax, bx) * beta_n(y, ay, by)
def beta(y, x):
    return beta_coef(y, x, 10, 10, 11, 9)

dblquad(beta, 0, 1, 0, lambda x: x)


In [ ]:

ps = 0.1 * np.random.rand(10, 1)
scores = []
num_noties = []
for _ in range(10000):
    match_notie = np.random.rand(*ps.shape) < ps
    num_noties.append(match_notie.sum())

    signs = np.sign(np.random.randn(*ps.shape))[match_notie > 0]
    s = signs.sum()
    scores.append(s)

plt.hist(scores, bins=50)
plt.figure()
plt.hist(num_noties, bins=20)


In [ ]:
!pip install py-irt


In [ ]:
!wget https://raw.githubusercontent.com/nd-ball/py-irt/d2a27dd55a84459782a5514e752ee48d9a63626e/test_fixtures/minitest.jsonlines
!cat minitest.jsonlines

!py-irt train 1pl minitest.jsonlines test-1pl/ --lr 0.02 --epochs 100


In [ ]:
import arena
import importlib
importlib.reload(arena)

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy.random as rng

tie_probs = np.concatenate((1 - 0.05 * np.random.rand(100), 0*np.random.rand(100)))
weights = rng.rand(tie_probs.size)
# print(tie_probs)

samps = []
for _ in range(1000):
    p = tie_probs.size
    response_a = (rng.rand(p) > tie_probs) * np.sign(rng.randn(p))
    response_b = response_a * -1
    response_b = np.sign(rng.randn(p))
    cdf, pvalue = arena.sign_test_niid(response_a, response_b, weights, tie_probs)
    samps.append(pvalue)

plt.hist(samps)

In [ ]:
print(cdf)
ax = plt.subplot()
cdf.plot(ax)
print(cdf.evaluate(-0.1))
print(cdf.evaluate(0.1))

In [ ]:
import arena
import importlib
importlib.reload(arena)
def trinomial(na, nb, n0):
    n = na + nb + n0
    cdf, pvalue = arena.sign_test_niid(([1]*na + [0]*nb + [0]*n0), np.array([0]*na + [1]*nb + [0]*n0), tie_probs=None, weights=None, sample_all=False)
    cdf, pvalue = arena.sign_test_niid(np.array([1]*na + [0]*nb + [0]*n0), np.array([0]*na + [1]*nb + [0]*n0), tie_probs=n0 / n * np.array([1] * n), weights=None, sample_all=True)
    print('binom', stats.binomtest(na, na + nb, p=0.5).pvalue)
    return pvalue

# trinomial(20, 12, 133)

def bootstrap_consistency(battles: pd.Series, num_round=1000, interpolation='nearest'):
    rows = []
    counts = Counter(battles)
    sign = np.sign(counts['model_a'] - counts['model_b'])
    for i in range(num_round):
        counts = Counter(battles.sample(frac=1.0, replace=True))
        diff = counts['model_a'] - counts['model_b']
        rows.append(diff)
    return 1 - np.mean(np.sign(rows) == sign)



print(bootstrap_ci(pd.Series(['model_a', 'model_b', 'model_a', 'both']*2)))
    
thres = stats.chi2.ppf(1-0.1, 1)
print(thres, np.mean(np.random.randn(100000)**2 > thres))

In [ ]:
print(stats.binomtest(61, 100, p=0.5).pvalue)
print(stats.binomtest(61, 100, p=0.5, alternative='greater').pvalue * 2)